# 42186 Model-based machine learning
# Modelling traffic accident injury severity

## Packages

First we read all the packages needed

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from tabulate import tabulate
from sklearn.metrics import make_scorer
from pathlib import Path
import torch
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
import os
from torch import nn
from pyro.nn import PyroModule
from pyro.nn.module import to_pyro_module_
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif

## Read data

Then we read our data, select the features that we want to use and do some preproceesing in order to get date information (hour, day of week and so on)

In [2]:
os.getcwd()

'/Users/frida/OneDrive - Danmarks Tekniske Universitet/Model based machine learning'

In [3]:
path = r'/Users/frida/OneDrive - Danmarks Tekniske Universitet/Model based machine learning/ACCIDENT/'

df_event = pd.read_csv(path + "ACCIDENT_EVENT.csv")
df_loc = pd.read_csv(path + "ACCIDENT_LOCATION.csv")
df_acc = pd.read_csv(path + "ACCIDENT.csv",low_memory=False)
df_veh = pd.read_csv(path + "VEHICLE.csv",low_memory=False)
df_person = pd.read_csv(path + "PERSON.csv",low_memory=False)

In [26]:
inter_cols = df_acc.columns.intersection(df_loc.columns)
inter_cols = [col for col in inter_cols]
data = pd.merge(df_acc, df_loc, on=inter_cols, how='inner')

inter_cols = data.columns.intersection(df_event.columns)
inter_cols = [col for col in inter_cols]
data = pd.merge(data, df_event, on=inter_cols, how='inner')
y = data['SEVERITY']

# Selecting the relevant features 
data = data[['ACCIDENTDATE', 'ACCIDENTTIME', 'ACCIDENT_TYPE', 'DAY_OF_WEEK', 'LIGHT_CONDITION', 
             'ROAD_GEOMETRY', 'SPEED_ZONE']]
data['ACCIDENTDATE'] = pd.to_datetime(data['ACCIDENTDATE'], format="%d/%m/%Y")
#data['ACCIDENTTIME'] = pd.to_datetime(data['ACCIDENTTIME'], format="%H:%M:%S")
data['MONTH'] = data['ACCIDENTDATE'].dt.month
data['YEAR'] = data['ACCIDENTDATE'].dt.year
data['HOUR'] = data['ACCIDENTTIME'].apply(lambda x: x[0:2])
data['LIGHT_CONDITION'] = data['LIGHT_CONDITION'].replace(9,7,regex=True)
data['DAY_OF_WEEK'] = data['DAY_OF_WEEK'].replace(0,1,regex=True)
# Making time features periodic
data['day_of_week_sin'] = np.sin(data['DAY_OF_WEEK'] * (2 * np.pi / 7))
data['day_of_week_cos'] = np.cos(data['DAY_OF_WEEK'] * (2 * np.pi / 7))
data['MONTH_sin'] = np.sin(data['MONTH']* (2 * np.pi / 12))
data['MONTH_cos'] = np.cos(data['MONTH']* (2 * np.pi / 12))

# Dropping features that have been modifed
data = data.drop(['ACCIDENTDATE', 'ACCIDENTTIME', 'MONTH', 'DAY_OF_WEEK'], axis = 1)

# Making SPEED_ZONE categorical
rep = {30:1 , 40:2, 50: 3, 60:4, 70:5, 75:6, 80:7, 90: 8, 100:9, 110:10, 777:11, 888:12, 999:13}
data['SPEED_ZONE'] = data['SPEED_ZONE'].replace(rep)

X = data
X

,ACCIDENT_TYPE,LIGHT_CONDITION,ROAD_GEOMETRY,SPEED_ZONE,YEAR,HOUR,day_of_week_sin,day_of_week_cos,MONTH_sin,MONTH_cos
0,1,1,1,4,2006,12,-7.818315e-01,0.623490,0.5,0.866025
1,1,1,2,5,2006,19,-7.818315e-01,0.623490,0.5,0.866025
2,7,1,5,9,2006,12,-2.449294e-16,1.000000,0.5,0.866025
3,1,1,2,7,2006,11,-2.449294e-16,1.000000,0.5,0.866025
4,1,1,5,3,2006,10,-2.449294e-16,1.000000,0.5,0.866025
...,...,...,...,...,...,...,...,...,...,...
326633,1,1,1,4,2020,18,7.818315e-01,0.623490,-0.5,0.866025
326634,6,1,5,7,2020,12,7.818315e-01,0.623490,-0.5,0.866025
326635,6,1,5,7,2020,12,7.818315e-01,0.623490,-0.5,0.866025
326636,4,3,5,7,2020,01,4.338837e-01,-0.900969,-0.5,0.866025


We will now use the Sclearn function SelectKBest in order to find the most important features to predict severity

In [25]:
bestfeatures = SelectKBest(score_func=mutual_info_classif, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score')) 


             Specs     Score
0    ACCIDENT_TYPE  0.025207
2    ROAD_GEOMETRY  0.024323
1  LIGHT_CONDITION  0.020074
3       SPEED_ZONE  0.018345
5             HOUR  0.008624
4             YEAR  0.007472
7  day_of_week_cos  0.006617
6  day_of_week_sin  0.004802
9        MONTH_cos  0.004348
8        MONTH_sin  0.001932


We choose the three most important features ROAD_GEOMETRY, ACCIDENT_TYPE, LIGHT_CONDITION

In [482]:
# Saving number of unique entries in the three most important features
n_acc = len(data_.ACCIDENT_TYPE.unique())
n_light = len(data_.LIGHT_CONDITION.unique())
n_road = len(data_.ROAD_GEOMETRY.unique())

Pivot tables for the chosen features vs. severity.

In [82]:
ls = data_.groupby(['SEVERITY','ROAD_GEOMETRY']).size().unstack()
ls.div(ls.sum(axis=0))

ROAD_GEOMETRY,1,2,3,4,5,6,7,8,9
SEVERITY,,,,,,,,,
1,0.012783,0.015909,0.008602,0.015688,0.031156,0.019231,NaN,NaN,0.008658
2,0.312064,0.342381,0.305376,0.335280,0.386767,0.413462,0.5,0.125,0.362193
3,0.675138,0.641696,0.686022,0.649032,0.582066,0.567308,0.5,0.875,0.629149
4,0.000015,0.000014,NaN,NaN,0.000011,NaN,NaN,NaN,NaN


In [84]:
ls = data_.groupby(['SEVERITY','ACCIDENT_TYPE']).size().unstack()
ls.div(ls.sum(axis=0))

ACCIDENT_TYPE,1,2,3,4,5,6,7,8,9
SEVERITY,,,,,,,,,
1,0.016114,0.034589,0.017334,0.044013,0.015033,0.019940,0.012464,0.007917,0.011429
2,0.309581,0.413287,0.353924,0.463689,0.364421,0.394203,0.400499,0.373021,0.337143
3,0.674300,0.552076,0.628743,0.492298,0.620546,0.585789,0.587038,0.618987,0.651429
4,0.000005,0.000048,NaN,NaN,NaN,0.000069,NaN,0.000075,NaN


In [85]:
ls = data_.groupby(['SEVERITY','LIGHT_CONDITION']).size().unstack()
ls.div(ls.sum(axis=0))

LIGHT_CONDITION,1,2,3,4,5,6,9
SEVERITY,,,,,,,
1,0.019888,0.018194,0.025267,0.039766,0.068714,0.015091,0.008672
2,0.344234,0.351983,0.412896,0.410526,0.457031,0.286720,0.181711
3,0.635869,0.629824,0.561800,0.549708,0.474255,0.698189,0.809618
4,0.000009,NaN,0.000037,NaN,NaN,NaN,NaN


For all the pivot tables we see a very clear distinction between the severity categories.

Next, we extract observations from out data set

In [483]:
numobs = 5000
X = data[['ACCIDENT_TYPE', 'LIGHT_CONDITION', 'ROAD_GEOMETRY']]
X = X[:numobs]
y = data_['SEVERITY']
y = y[:numobs]
X = np.array(X)
y = np.array(y)
ind = np.arange(0,numobs)

We then prepare our test and training sets 

In [562]:
train_perc = 0.66 # percentage of training data

# Large data set
split_point = int(train_perc*len(y))
perm = np.random.permutation(len(y))
ix_train = perm[:split_point]
ix_test = perm[split_point:]
X_train = X[ix_train,:]
X_test = X[ix_test,:]
ind_train = ind[ix_train]
ind_test = ind[ix_test]
y_train = y[ix_train]
y_test = y[ix_test]
print("num train: %d" % len(y_train))
print("num test: %d" % len(y_test))

X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
ind_train = torch.tensor(ind_train).long()


num train: 3300
num test: 1700


We then define our hierarchical model 

In [646]:
def hierarchical_model(X, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu, obs=None):
 
    input_dim = X.shape[1]
    
    mu_c = pyro.sample("mu_c", dist.Normal(torch.zeros(n_cat), 
                                                       lambda_val*torch.ones(n_cat)).to_event())# Prior for the bias mean
        

    sigma_c  = pyro.sample("sigma_c",  dist.HalfCauchy(tau*torch.ones(n_cat)).to_event()) # Prior for the bias standard deviation
    beta  = pyro.sample("beta", dist.Normal(torch.zeros(input_dim, n_cat), 
                                            nu*torch.ones(input_dim, n_cat)).to_event()) # Priors for the regression coefficents

    with pyro.plate("acc", n_acc):
        with pyro.plate("light", n_light):
            with pyro.plate("road", n_road):
                alpha = pyro.sample("alpha", dist.Normal(mu_c, sigma_c).to_event(1)) # Draw the individual parameter for each individual

    with pyro.plate("data", X.shape[0]):
        acc = np.array(X[:,0])
        light = np.array(X[:,1])
        road = np.array(X[:,2])
        logits = alpha[acc-1,light-1,road-1] + X.matmul(beta)
        y = pyro.sample("y", dist.Categorical(logits=logits), obs=obs) # If you use logits you don't need to do sigmoid
        
    return y

Preparing our model. These are simply guesses for the prior values.

In [647]:
n_cat = 4 #number of severity categories
indx = numobs
tau = 10
lambda_val= 10
nu = 7

Training our model using gradient step.

In [576]:
%%time
#Train the herarical model without the ANN on the X*beta part 
# Define guide function
guide = AutoDiagonalNormal(hierarchical_model)

# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 12000
 
    
# Setup the optimizer
adam_params = {"lr": 0.005}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=3)
svi = SVI(hierarchical_model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu , y_train-1)
    if step % 500 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 40593.3
[500] ELBO: 5232.1
[1000] ELBO: 3056.0
[1500] ELBO: 2856.4
[2000] ELBO: 2808.2
[2500] ELBO: 2802.0
[3000] ELBO: 2788.3
[3500] ELBO: 2777.0
[4000] ELBO: 2778.5
[4500] ELBO: 2768.8
[5000] ELBO: 2763.4
[5500] ELBO: 2759.9
[6000] ELBO: 2752.4
[6500] ELBO: 2756.7
[7000] ELBO: 2742.6
[7500] ELBO: 2761.6
[8000] ELBO: 2818.6
[8500] ELBO: 2746.1
[9000] ELBO: 2740.8
[9500] ELBO: 2741.2
[10000] ELBO: 2742.9
[10500] ELBO: 2739.0
[11000] ELBO: 2737.6
[11500] ELBO: 2729.4
CPU times: user 3min 16s, sys: 4.72 s, total: 3min 20s
Wall time: 3min 25s


In [559]:
predictive = Predictive(hierarchical_model, guide=guide, num_samples=2000,
                        return_sites=("beta", "alpha", "mu_c", "sigma_c"))
samples = predictive(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu , y_train-1)

We then extract expected values of the parameters

In [560]:
alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

and make predictions for test set and compute the accuracy

In [561]:
acc = X_test[:,0]
light = X_test[:,1]
road =  X_test[:,2]
y_hat = alpha_hat[acc-1,light-1,road-1,:] + np.squeeze(np.dot(X_test, beta_hat[0]))
y_hat = np.argmax(y_hat, axis=1) + 1
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [3 3 3 ... 3 3 3]
true values: [2 2 2 ... 2 2 2]
Accuracy: 0.5658823529411765


We obtain an accuracy on 56.5 %

# Tuning priors

In order to optimize we tune the priors by computing the accuracy for all combinations of the ranges stated.

In [551]:
# Ranges
l_v = [8,10,12]
t_v = [8,10,12]
n_c = [6,8,10]

acc_mat = np.zeros((len(l_v),len(t_v),len(n_c)))
l=0
for lambda_val in l_v:
    t = 0
    for tau in t_v:
        b=0
        for nu in n_c:
            
            # Define guide function
            guide = AutoDiagonalNormal(hierarchical_model)

            # Reset parameter values
            pyro.clear_param_store()

            # Define the number of optimization steps
            n_steps = 2500

            # Setup the optimizer
            adam_params = {"lr": 0.005}
            optimizer = ClippedAdam(adam_params)

            # Setup the inference algorithm
            elbo = Trace_ELBO(num_particles=3)
            svi = SVI(hierarchical_model, guide, optimizer, loss=elbo)

            # Do gradient steps
            for step in range(n_steps):
                elbo = svi.step(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu, y_train-1)
                
            predictive = Predictive(hierarchical_model, guide=guide, num_samples=2000,
                                    return_sites=("beta", "alpha", "alpha_mu", "alpha_sigma"))
            samples = predictive(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu, y_train-1)
            
            # extract expected values of the parameters
            alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
            beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

            # make predictions for test set
            acc = X_test[:,0]
            light = X_test[:,1]
            road =  X_test[:,2]
            y_hat = alpha_hat[acc-1,light-1,road-1,:] + np.squeeze(np.dot(X_test, beta_hat[0]))
            y_hat = np.argmax(y_hat, axis=1) + 1

            # evaluate prediction accuracy
            acc_mat[l,t,b] = 1.0*np.sum(y_hat == y_test) / len(y_test)
            print("acc for l=",lambda_val, "t=",tau, "nu=",nu, "acc = ", acc_mat[l,t,b])
            
            b+=1
        t+=1
    l+=1


acc for l= 8 t= 8 nu= 6 acc =  0.558235294117647
acc for l= 8 t= 8 nu= 8 acc =  0.5623529411764706
acc for l= 8 t= 8 nu= 10 acc =  0.5623529411764706
acc for l= 8 t= 10 nu= 6 acc =  0.5605882352941176
acc for l= 8 t= 10 nu= 8 acc =  0.5517647058823529
acc for l= 8 t= 10 nu= 10 acc =  0.5658823529411765
acc for l= 8 t= 12 nu= 6 acc =  0.5594117647058824
acc for l= 8 t= 12 nu= 8 acc =  0.5517647058823529
acc for l= 8 t= 12 nu= 10 acc =  0.5617647058823529
acc for l= 10 t= 8 nu= 6 acc =  0.5629411764705883
acc for l= 10 t= 8 nu= 8 acc =  0.5529411764705883
acc for l= 10 t= 8 nu= 10 acc =  0.5670588235294117
acc for l= 10 t= 10 nu= 6 acc =  0.5641176470588235
acc for l= 10 t= 10 nu= 8 acc =  0.5505882352941176
acc for l= 10 t= 10 nu= 10 acc =  0.5617647058823529
acc for l= 10 t= 12 nu= 6 acc =  0.5605882352941176
acc for l= 10 t= 12 nu= 8 acc =  0.5641176470588235
acc for l= 10 t= 12 nu= 10 acc =  0.5535294117647059
acc for l= 12 t= 8 nu= 6 acc =  0.55
acc for l= 12 t= 8 nu= 8 acc =  0.56


In [553]:
np.max(acc_mat)

0.5670588235294117

This corresponds to lambda = 12 tau = 10 nu = 10 with an accuracy of 56.7%. Using this information we try to train the model for 12000 steps using these priors.

In [577]:
%%time
#Train the herarical model without the ANN on the X*beta part 
# Define guide function
guide = AutoDiagonalNormal(hierarchical_model)

# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 12000
 
    
# Setup the optimizer
adam_params = {"lr": 0.005}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=3)
svi = SVI(hierarchical_model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X_train, n_cat, n_acc, n_light, n_road, 12, 10, 10, y_train-1)
    if step % 500 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 32589.4
[500] ELBO: 7034.4
[1000] ELBO: 3801.8
[1500] ELBO: 3225.0
[2000] ELBO: 3007.6
[2500] ELBO: 2953.1
[3000] ELBO: 2880.7
[3500] ELBO: 2857.7
[4000] ELBO: 2841.7
[4500] ELBO: 2817.0
[5000] ELBO: 2819.6
[5500] ELBO: 2811.2
[6000] ELBO: 2812.7
[6500] ELBO: 2793.1
[7000] ELBO: 2790.4
[7500] ELBO: 2788.6
[8000] ELBO: 2791.1
[8500] ELBO: 2794.0
[9000] ELBO: 2782.9
[9500] ELBO: 2772.8
[10000] ELBO: 2783.7
[10500] ELBO: 2769.8
[11000] ELBO: 2770.6
[11500] ELBO: 2768.0
CPU times: user 3min 22s, sys: 4.95 s, total: 3min 27s
Wall time: 3min 37s


In [578]:
predictive = Predictive(hierarchical_model, guide=guide, num_samples=2000,
                        return_sites=("beta", "alpha", "mu_c", "sigma_c"))
samples = predictive(X_train, n_cat, n_acc, n_light, n_road, 12, 10, 10, y_train-1)

alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

acc = X_test[:,0]
light = X_test[:,1]
road =  X_test[:,2]
y_hat = alpha_hat[acc-1,light-1,road-1,:] + np.squeeze(np.dot(X_test, beta_hat[0]))
y_hat = np.argmax(y_hat, axis=1) + 1
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [3 3 2 ... 3 3 3]
true values: [3 2 2 ... 2 2 3]
Accuracy: 0.5747058823529412


Here we improve our accuracy by 1%.

The next thing we tried was to use individual priors for the mean and sd. for each of the categories. Here we due to long computation time tuned the parameters by for each element in each prior, incrementing the given element by one, computing the accuracy. If incrementing the element increased the accuracy we keep it and if not we stopped tuning this element and moved to tuning the next element in the prior vector. We start by assuming that they are = [1,1,1,1], we then increment each element one at a time and if it doesn't increasethe accuracy with more than 0.5% we move on. 

The drawback with this method is that we don't test all combinations of value choices, but due to long computation times we assessed that this would be too time consuming. 

In [566]:
lambda_val = torch.tensor([1,1,1,1])
tau_val = 10
nu_val = 7

n_steps = 100
acc_mat = np.zeros(n_steps - 1)

j = 0
acc_mat[0] = 0
for i in range(1,n_steps):
    
    # Define guide function
    guide = AutoDiagonalNormal(hierarchical_model)

    # Reset parameter values
    pyro.clear_param_store()

    # Define the number of optimization steps
    n_steps = 2500

    # Setup the optimizer
    adam_params = {"lr": 0.005}
    optimizer = ClippedAdam(adam_params)

    # Setup the inference algorithm
    elbo = Trace_ELBO(num_particles=3)
    svi = SVI(hierarchical_model, guide, optimizer, loss=elbo)

    # Do gradient steps
    for step in range(n_steps):
        elbo = svi.step(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu_val, y_train-1)

    predictive = Predictive(hierarchical_model, guide=guide, num_samples=2000,
                            return_sites=("beta", "alpha", "alpha_mu", "alpha_sigma"))
    samples = predictive(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau_val, nu_val, y_train-1)
    # extract expected values of the parameters
    alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
    beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

    # make predictions for test set
    acc = X_test[:,0]
    light = X_test[:,1]
    road =  X_test[:,2]
    y_hat = alpha_hat[acc-1,light-1,road-1,:] + np.squeeze(np.dot(X_test, beta_hat[0]))
    y_hat = np.argmax(y_hat, axis=1) + 1

    # evaluate prediction accuracy
    acc_mat[i] = 1.0*np.sum(y_hat == y_test) / len(y_test)
    print("acc for l=",lambda_val, "t=",tau_val, "nu=",nu_val, "acc = ", acc_mat[i])
    
    if (acc_mat[i-1] - 0.005 > acc_mat[i]) & (j < 3):
        lambda_val[j] = lambda_val[j] - 1
        acc_mat[i] = acc_mat[i-1]
        j += 1
    if (acc_mat[i-1] - 0.005 > acc_mat[i]) & (j == 3):
        break
    
    lambda_val[j] += 1


acc for l= tensor([1, 1, 1, 1]) t= 10 nu= 7 acc =  0.5694117647058824
acc for l= tensor([2, 1, 1, 1]) t= 10 nu= 7 acc =  0.5676470588235294
acc for l= tensor([3, 1, 1, 1]) t= 10 nu= 7 acc =  0.5705882352941176
acc for l= tensor([4, 1, 1, 1]) t= 10 nu= 7 acc =  0.5752941176470588
acc for l= tensor([5, 1, 1, 1]) t= 10 nu= 7 acc =  0.5794117647058824
acc for l= tensor([6, 1, 1, 1]) t= 10 nu= 7 acc =  0.5723529411764706
acc for l= tensor([5, 2, 1, 1]) t= 10 nu= 7 acc =  0.5711764705882353
acc for l= tensor([5, 1, 2, 1]) t= 10 nu= 7 acc =  0.5711764705882353
acc for l= tensor([5, 1, 1, 2]) t= 10 nu= 7 acc =  0.5776470588235294
acc for l= tensor([5, 1, 1, 3]) t= 10 nu= 7 acc =  0.5705882352941176


In [626]:
lambda_val = torch.tensor([5, 1, 1, 1])
tau_val = torch.tensor([1,1,1,1])
nu_val = 7

n_steps = 100
acc_mat = np.zeros(n_steps - 1)

j = 0
acc_mat[0] = 0
for i in range(1,n_steps):
    
    # Define guide function
    guide = AutoDiagonalNormal(hierarchical_model)

    # Reset parameter values
    pyro.clear_param_store()

    # Define the number of optimization steps
    n_steps = 2500

    # Setup the optimizer
    adam_params = {"lr": 0.005}
    optimizer = ClippedAdam(adam_params)

    # Setup the inference algorithm
    elbo = Trace_ELBO(num_particles=3)
    svi = SVI(hierarchical_model, guide, optimizer, loss=elbo)

    # Do gradient steps
    for step in range(n_steps):
        elbo = svi.step(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu_val, y_train-1)

    predictive = Predictive(hierarchical_model, guide=guide, num_samples=2000,
                            return_sites=("beta", "alpha", "alpha_mu", "alpha_sigma"))
    samples = predictive(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau_val, nu_val, y_train-1)
    # extract expected values of the parameters
    alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
    beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

    # make predictions for test set
    acc = X_test[:,0]
    light = X_test[:,1]
    road =  X_test[:,2]
    y_hat = alpha_hat[acc-1,light-1,road-1,:] + np.squeeze(np.dot(X_test, beta_hat[0]))
    y_hat = np.argmax(y_hat, axis=1) + 1

    # evaluate prediction accuracy
    acc_mat[i] = 1.0*np.sum(y_hat == y_test) / len(y_test)
    print("acc for l=",lambda_val, "t=",tau_val, "nu=",nu_val, "acc = ", acc_mat[i])
    
    if (acc_mat[i-1] - 0.005 > acc_mat[i]) & (j < 3):
        tau_val[j] = tau_val[j] - 1
        acc_mat[i] = acc_mat[i-1]
        j += 1
    if (acc_mat[i-1] - 0.005 > acc_mat[i]) & (j == 3):
        break
    
    tau_val[j] += 1


acc for l= tensor([5, 1, 1, 1]) t= tensor([1, 1, 1, 1]) nu= 7 acc =  0.5670588235294117
acc for l= tensor([5, 1, 1, 1]) t= tensor([2, 1, 1, 1]) nu= 7 acc =  0.5752941176470588
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 1, 1, 1]) nu= 7 acc =  0.5758823529411765
acc for l= tensor([5, 1, 1, 1]) t= tensor([4, 1, 1, 1]) nu= 7 acc =  0.5682352941176471
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 2, 1, 1]) nu= 7 acc =  0.5788235294117647
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 3, 1, 1]) nu= 7 acc =  0.5741176470588235
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 4, 1, 1]) nu= 7 acc =  0.571764705882353
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 5, 1, 1]) nu= 7 acc =  0.5770588235294117
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6, 1, 1]) nu= 7 acc =  0.58
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 7, 1, 1]) nu= 7 acc =  0.5723529411764706
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6, 2, 1]) nu= 7 acc =  0.5729411764705883
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6

In [648]:
lambda_val = torch.tensor([5, 1, 1, 1])
tau_val = torch.tensor([3, 6, 1, 1])
nu_val = torch.tensor([1,1,1,1])

n_steps = 100
acc_mat = np.zeros(n_steps - 1)

j = 0
acc_mat[0] = 0
for i in range(1,n_steps):
    
    # Define guide function
    guide = AutoDiagonalNormal(hierarchical_model)

    # Reset parameter values
    pyro.clear_param_store()

    # Define the number of optimization steps
    n_steps = 2500

    # Setup the optimizer
    adam_params = {"lr": 0.005}
    optimizer = ClippedAdam(adam_params)

    # Setup the inference algorithm
    elbo = Trace_ELBO(num_particles=3)
    svi = SVI(hierarchical_model, guide, optimizer, loss=elbo)

    # Do gradient steps
    for step in range(n_steps):
        elbo = svi.step(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu_val, y_train-1)

    predictive = Predictive(hierarchical_model, guide=guide, num_samples=2000,
                            return_sites=("beta", "alpha", "alpha_mu", "alpha_sigma"))
    samples = predictive(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau_val, nu_val, y_train-1)
    # extract expected values of the parameters
    alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
    beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

    # make predictions for test set
    acc = X_test[:,0]
    light = X_test[:,1]
    road =  X_test[:,2]
    y_hat = alpha_hat[acc-1,light-1,road-1,:] + np.squeeze(np.dot(X_test, beta_hat[0]))
    y_hat = np.argmax(y_hat, axis=1) + 1

    # evaluate prediction accuracy
    acc_mat[i] = 1.0*np.sum(y_hat == y_test) / len(y_test)
    print("acc for l=",lambda_val, "t=",tau_val, "nu=",nu_val, "acc = ", acc_mat[i])
    
    if (acc_mat[i-1] - 0.005 > acc_mat[i]) & (j < 3):
        nu_val[j] = nu_val[j] - 1
        acc_mat[i] = acc_mat[i-1]
        j += 1
    if (acc_mat[i-1] - 0.005 > acc_mat[i]) & (j == 3):
        break
    
    nu_val[j] += 1


acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6, 1, 1]) nu= tensor([1, 1, 1, 1]) acc =  0.5641176470588235
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6, 1, 1]) nu= tensor([2, 1, 1, 1]) acc =  0.571764705882353
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6, 1, 1]) nu= tensor([3, 1, 1, 1]) acc =  0.5670588235294117
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6, 1, 1]) nu= tensor([4, 1, 1, 1]) acc =  0.5758823529411765
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6, 1, 1]) nu= tensor([5, 1, 1, 1]) acc =  0.5682352941176471
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6, 1, 1]) nu= tensor([4, 2, 1, 1]) acc =  0.5705882352941176
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6, 1, 1]) nu= tensor([4, 1, 2, 1]) acc =  0.5664705882352942
acc for l= tensor([5, 1, 1, 1]) t= tensor([3, 6, 1, 1]) nu= tensor([4, 1, 1, 2]) acc =  0.5670588235294117


Here the maximum accuracy is found when nu = [4, 1, 1, 1], however we get an even higher accuracy when nu is 1-dimational i.e. for lambda = [5, 1, 1, 1], tau = [3, 6, 1, 1] and nu = 7  we get an accuracy of 58%, this is therefore our optimal choice.

# Extending the model with FFNN

Now we try to use a feed forward neural network in our hierarchical model. We wish to use this FFNN on the  $X\beta$ part of the funtion that we use in the categorical distribution.

In [611]:
class FFNN(torch.nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super(FFNN, self).__init__()
        
        # Architecture
        self.in_layer = torch.nn.Linear(n_in, n_hidden)
        self.h_layer = torch.nn.Linear(n_hidden, n_hidden)
        self.out_layer = torch.nn.Linear(n_hidden, n_out)
        
        # Activation functions
        self.tanh = torch.nn.Tanh()
        
    def forward(self, X):
        # Forward pass
        X = self.tanh(self.in_layer(X))
        X = self.tanh(self.h_layer(X))
        X = self.out_layer(X)
        
        return X

In [612]:
def hierarchical_model(X, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu, obs=None):
    torch_model = FFNN(n_in=3, n_hidden=3, n_out=n_cat)
    
    input_dim = X.shape[1]
    
    mu_c = pyro.sample("mu_c", dist.Normal(torch.zeros(n_cat), 
                                                       lambda_val*torch.ones(n_cat)).to_event())# Prior for the bias mean
        

    sigma_c  = pyro.sample("sigma_c",  dist.HalfCauchy(tau*torch.ones(n_cat)).to_event()) # Prior for the bias standard deviation
    beta  = pyro.sample("beta", dist.Normal(torch.zeros(input_dim, n_cat), 
                                            nu*torch.ones(input_dim, n_cat)).to_event()) # Priors for the regression coefficents
   
    bayesian_model =pyro.random_module('bayesian_model', torch_model, [mu_c,sigma_c,beta]) # Make this model and these priors a Pyro model
    sampled_model = bayesian_model() 
    
    with pyro.plate("acc", n_acc):
        with pyro.plate("light", n_light):
            with pyro.plate("road", n_road):
                alpha = pyro.sample("alpha", dist.Normal(mu_c, sigma_c).to_event(1)) # Draw the individual parameter for each individual

    with pyro.plate("data", X.shape[0]):
        acc = np.array(X[:,0])
        light = np.array(X[:,1])
        road = np.array(X[:,2])
        nn1 = sampled_model(X.matmul(beta)).squeeze(-1)
        logits = alpha[acc-1,light-1,road-1] + nn1 
        y = pyro.sample("y", dist.Categorical(logits=logits), obs=obs) # If you use logits you don't need to do sigmoid
        
    return y

Because there are very few observaions in the 4'th category we now assume that there are only 3 categories. When doing this we saw that the model with FFNN performed better than with 4 categories.

In [653]:
# Prepare data for Pyro model with priors for each category
tau = 10
lambda_val= 12
nu= 10
n_cat = 3 

In [496]:
%%time
#Train the herarical model with ANN


# Define guide function
guide = AutoDiagonalNormal(hierarchical_model)

# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 12000

# Setup the optimizer
adam_params = {"lr": 0.005}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=3)
svi = SVI(hierarchical_model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X_train, n_cat,n_acc, n_light, n_road, lambda_val, tau, ny, y_train-1)
    if step % 500 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

/Users/frida/opt/anaconda3/lib/python3.8/site-packages/pyro/primitives.py:491: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  warnings.warn(


[0] ELBO: 15040.0
[500] ELBO: 4708.3
[1000] ELBO: 3093.0
[1500] ELBO: 2873.0
[2000] ELBO: 2833.1
[2500] ELBO: 2795.4
[3000] ELBO: 2773.3
[3500] ELBO: 2761.4
[4000] ELBO: 2749.4
[4500] ELBO: 2729.0
[5000] ELBO: 2726.9
[5500] ELBO: 2718.7
[6000] ELBO: 2702.1
[6500] ELBO: 2690.5
[7000] ELBO: 2692.1
[7500] ELBO: 2681.2
[8000] ELBO: 2690.7
[8500] ELBO: 2680.0
[9000] ELBO: 2685.2
[9500] ELBO: 2669.1
[10000] ELBO: 2666.1
[10500] ELBO: 2673.4
[11000] ELBO: 2666.2
[11500] ELBO: 2665.4
CPU times: user 4min 11s, sys: 5.44 s, total: 4min 16s
Wall time: 4min 23s


In [618]:
from pyro.infer import Predictive

predictive = Predictive(hierarchical_model, guide=guide, num_samples=2000,
                        return_sites=("beta", "alpha", "mu_c", "sigma_c"))
samples = predictive(X_train, n_cat,n_acc, n_light, n_road, lambda_val, tau, nu, y_train-1)


# extract expected values of the parameters
alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

torch_model = FFNN(n_in=3, n_hidden=3, n_out=n_cat)
tX_test = torch.tensor(X_test).float()
# make predictions for test set
acc = X_test[:,0]
light = X_test[:,1]
road =  X_test[:,2]
yn = alpha_hat[acc-1,light-1,road-1,:]  + np.squeeze(torch_model(tX_test.matmul(torch.tensor(beta_hat[0]).float())).detach().numpy())
yn = np.argmax(yn, axis=1) + 1
print("predictions:", yn)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(1.0*np.sum(yn == y_test) / len(y_test)))

predictions: [2 2 2 ... 2 2 2]
true values: [3 2 2 ... 2 2 3]
Accuracy: 0.42058823529411765


We here obtain an accuracy of 42%.

We now tune the priors like previously, the ranges chosen are shown below.

In [615]:
# Ranges
l_v = [8, 9, 10, 11, 12]
t_v = [4, 5, 6, 7]
n_c = [11, 12, 13, 14]

acc_mat = np.zeros((len(l_v),len(t_v),len(n_c)))
l=0
for lambda_val in l_v:
    t = 0
    for tau in t_v:
        b=0
        for nu in n_c:
            
            # Define guide function
            guide = AutoDiagonalNormal(hierarchical_model)

            # Reset parameter values
            pyro.clear_param_store()

            # Define the number of optimization steps
            n_steps = 2500

            # Setup the optimizer
            adam_params = {"lr": 0.005}
            optimizer = ClippedAdam(adam_params)

            # Setup the inference algorithm
            elbo = Trace_ELBO(num_particles=3)
            svi = SVI(hierarchical_model, guide, optimizer, loss=elbo)

            # Do gradient steps
            for step in range(n_steps):
                elbo = svi.step(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu, y_train-1)
                
            predictive = Predictive(hierarchical_model, guide=guide, num_samples=2000,
                                    return_sites=("beta", "alpha", "alpha_mu", "alpha_sigma"))
            samples = predictive(X_train, n_cat, n_acc, n_light, n_road, lambda_val, tau, nu, y_train-1)
            
            # extract expected values of the parameters
            alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
            beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

            torch_model = FFNN(n_in=3, n_hidden=3, n_out=n_cat)
            tX_test = torch.tensor(X_test).float()
            # make predictions for test set
            acc = X_test[:,0]
            light = X_test[:,1]
            road =  X_test[:,2]
            yn = alpha_hat[acc-1,light-1,road-1,:]  + np.squeeze(torch_model(tX_test.matmul(torch.tensor(beta_hat[0]).float())).detach().numpy())
            yn = np.argmax(yn, axis=1) + 1

            # evaluate prediction accuracy
            acc_mat[l,t,b] = 1.0*np.sum(1.0*np.sum(yn == y_test) / len(y_test))
            print("acc for l=",lambda_val, "t=",tau, "nu =",nu, "acc = ", acc_mat[l,t,b])
            
            b+=1
        t+=1
    l+=1


acc for l= 8 t= 4 nu = 11 acc =  0.42058823529411765
acc for l= 8 t= 4 nu = 12 acc =  0.42058823529411765
acc for l= 8 t= 4 nu = 13 acc =  0.4676470588235294
acc for l= 8 t= 4 nu = 14 acc =  0.03411764705882353
acc for l= 8 t= 5 nu = 11 acc =  0.5452941176470588
acc for l= 8 t= 5 nu = 12 acc =  0.03411764705882353
acc for l= 8 t= 5 nu = 13 acc =  0.42058823529411765
acc for l= 8 t= 5 nu = 14 acc =  0.31941176470588234
acc for l= 8 t= 6 nu = 11 acc =  0.5452941176470588
acc for l= 8 t= 6 nu = 12 acc =  0.03411764705882353
acc for l= 8 t= 6 nu = 13 acc =  0.03411764705882353
acc for l= 8 t= 6 nu = 14 acc =  0.03411764705882353
acc for l= 8 t= 7 nu = 11 acc =  0.36411764705882355
acc for l= 8 t= 7 nu = 12 acc =  0.03411764705882353
acc for l= 8 t= 7 nu = 13 acc =  0.42058823529411765
acc for l= 8 t= 7 nu = 14 acc =  0.42058823529411765
acc for l= 9 t= 4 nu = 11 acc =  0.42058823529411765
acc for l= 9 t= 4 nu = 12 acc =  0.1935294117647059
acc for l= 9 t= 4 nu = 13 acc =  0.467647058823529

In [616]:
np.max(acc_mat)

0.5629411764705883

We then train the model with lambda = 10 tau = 6 nu = 12.

In [620]:
%%time
#Train the herarical model with ANN


# Define guide function
guide = AutoDiagonalNormal(hierarchical_model)

# Reset parameter values
pyro.clear_param_store()

# Define the number of optimization steps
n_steps = 2500

# Setup the optimizer
adam_params = {"lr": 0.005}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm
elbo = Trace_ELBO(num_particles=3)
svi = SVI(hierarchical_model, guide, optimizer, loss=elbo)

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(X_train, n_cat,n_acc, n_light, n_road, 10, 6, 12, y_train-1)
    if step % 500 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

[0] ELBO: 15618.1
[500] ELBO: 3343.5
[1000] ELBO: 2767.9
[1500] ELBO: 2728.4
[2000] ELBO: 2710.0
CPU times: user 50.9 s, sys: 807 ms, total: 51.7 s
Wall time: 52.3 s


In [621]:
from pyro.infer import Predictive

predictive = Predictive(hierarchical_model, guide=guide, num_samples=2000,
                        return_sites=("beta", "alpha", "mu_c", "sigma_c"))
samples = predictive(X_train, n_cat,n_acc, n_light, n_road, 10, 6, 12, y_train-1)


# extract expected values of the parameters
alpha_hat = samples["alpha"].mean(axis=0).detach().numpy()
beta_hat = samples["beta"][:,0].mean(axis=0).detach().numpy()

torch_model = FFNN(n_in=3, n_hidden=3, n_out=n_cat)
tX_test = torch.tensor(X_test).float()
# make predictions for test set
acc = X_test[:,0]
light = X_test[:,1]
road =  X_test[:,2]
yn = alpha_hat[acc-1,light-1,road-1,:]  + np.squeeze(torch_model(tX_test.matmul(torch.tensor(beta_hat[0]).float())).detach().numpy())
yn = np.argmax(yn, axis=1) + 1
print("predictions:", yn)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(1.0*np.sum(yn == y_test) / len(y_test)))

predictions: [3 3 3 ... 3 3 3]
true values: [3 2 2 ... 2 2 3]
Accuracy: 0.5452941176470588


Here we obtain an accuracy of 54.5 %